### Throwing All Functions Together In One Spot: 

Testing & exploration can be found here: https://github.com/dwanneruchi/538_Riddlers/blob/master/20210723/riddler_express.ipynb

In [1]:
from itertools import product
import random

# Movement funcs:
def bishopMove(row, col):
    """Bishop can move diagonal 1 space"""
    return list(product([row-1, row+1],[col-1, col+1]))

def rookMove(row, col):
    """Rook can move horiz/vert 1 space"""
    moves = []
    for i,j in [(-1,0), (1,0),(0,-1), (0,1)]:
        moves.append((row + i, col + j))
    return moves

def knightMove(row, col):
    """Knight can move 2, 1 per usual (l-shape) -> 8 total moves
    Stack tips: https://stackoverflow.com/questions/19372622/how-do-i-generate-all-of-a-knights-moves
    """
    return list(product([row-1, row+1],[col-2, col+2])) + list(product([row-2,row+2],[col-1,col+1]))

def pawnMove(row, col):
    """Pawn can only move up-diagonal"""
    moves = []
    for i,j in [(-1,-1), (-1,1)]:
        moves.append((row + i, col + j))
    return moves

def queenMove(row, col):
    """Queen can move one space in any direction"""
    all_moves = list(product([row-1, row+1, row],[col-1, col+1, col]))
    
    return [move for move in all_moves if (move[0] != row or move[1] != col)] # need to ensure some move occurred

def move(p, row, col):
    """Determine current piece & return list of moves (not yet filtered)"""
    if p == 'b':
        return bishopMove(row, col)
    elif p == 'r':
        return rookMove(row, col)
    elif p == 'n':
        return knightMove(row, col)
    elif p == 'p':
        return pawnMove(row, col)
    elif p == 'q':
        return queenMove(row, col)
    else:
        raise TypeError("Not correct piece")

# Filtering out moves OR finding the king      
def checkMoveBoundary(moveList):
    """Check if any move goes outside boundary, if so pass"""
    return [(x,y) for x,y in moveList if x >= 0 and y >= 0 and x < 8 and y < 8]
    
def checkMoveBlack(moveList, gameBoard):
    """Check if any move leads to a black piece; remove as option"""
    return [(x,y) for x,y in moveList if gameBoard[x][y] != 'BL']
    
def checkMoveKing(moveList, gameBoard):
    """Check if we can move to a king (which results in a win)"""
    try:
        return [(x,y) for x,y in moveList if gameBoard[x][y] == 'K'][0]
    except:
        return False

# General Game Board
game_board = [
     ['K', 'BL', 'b', 'BL', 'K', 'r', 'b', 'r'],
     ['n', 'r', 'n', 'n', 'p', 'n', 'K', 'b'],
     ['r', 'BL', 'n', 'BL', 'p', 'r', 'p', 'r'],
     ['BL', 'BL', 'p', 'r', 'BL', 'n', 'BL', 'BL'],
     ['n', 'n', 'n', 'b', 'BL', 'b', 'r', 'b'],
     ['q', 'r', 'n', 'p', 'BL', 'n', 'r', 'q'],
     ['BL', 'BL', 'r', 'p', 'b', 'p', 'b', 'q'],
     ['K', 'b', 'q', 'n', 'p', 'r', 'n', 'n']
]

### Running Various Games

This could be more intelligently run, but I am running a series of games until one of the following occur:

- King is moved to (ends the full simulation)
- No eligible moves (ends the current game)

More intelligent approach: Keep track of sequences and determine some sequence length to not allow repeated...there is a non-zero change right now of being stuck in an infinite loop of moves. 

In [2]:
winner = False

for _ in range(1000):
    
    # before we overwrite sequence let's look at winner; if winner end sim
    if winner:
        break
    
    sequence = [] # keep track of order
    pos = (6,4)

    while True:
        # add move location to sequence
        sequence.append(pos)

        # determine piece & moves
        piece = game_board[pos[0]][pos[1]]
        pos_moves = move(piece, pos[0], pos[1])

        # filter out incorrect moves
        pos_moves = checkMoveBoundary(pos_moves)
        pos_moves = checkMoveBlack(pos_moves,game_board)

        # ensure we have moves left - if not we end our game & go to the next
        if len(pos_moves) == 0:
            break

        # Any kings? If so, we end the simulation 
        finish = checkMoveKing(pos_moves,game_board)

        if finish != False:
            print(f"Game over - found king at {finish}")
            winner = True
            break

        # If not, we need to choose a move from candidate moves: shuffle (Future: Check if we have last n moves in `sequence`)
        random.shuffle(pos_moves)

        pos = pos_moves[0]

Game over - found king at (0, 0)


In [3]:
sequence

[(6, 4),
 (5, 3),
 (4, 2),
 (6, 3),
 (5, 2),
 (6, 4),
 (5, 3),
 (4, 2),
 (5, 0),
 (5, 1),
 (4, 1),
 (2, 0),
 (1, 0),
 (0, 2),
 (1, 1),
 (1, 2)]

### Found A Differenct Sequence (Quicker) In a Prior Run: 

```
[(6, 4), 
 (5, 3),
 (4, 2),
 (5, 0),
 (4, 1),
 (2, 0),
 (1, 0),
 (0, 2),
 (1, 1),
 (1, 2),
 (0,0)]
```

This becomes: 

`Bishop -> Pawn -> Knight -> Queen -> Knight -> Rook -> Knight -> Bishop -> Rook -> Knight -> KING at (0,0)`

### Are There Multiple Solutions? 

Can we get to more than one king? 

Solution: Found 3 of the 4 have paths.

In [17]:
king_solution = {}

winner = False

for _ in range(100_000):
    
    # before we overwrite sequence let's look at winner; if winner end sim
    if winner:
        break
    
    sequence = [] # keep track of order
    pos = (6,4)
    i = 1 # let's kill it after 2K moves...arbitrary

    while True:
        # add move location to sequence
        sequence.append(pos)

        # determine piece & moves
        piece = game_board[pos[0]][pos[1]]
        pos_moves = move(piece, pos[0], pos[1])

        # filter out incorrect moves
        pos_moves = checkMoveBoundary(pos_moves)
        pos_moves = checkMoveBlack(pos_moves,game_board)

        # ensure we have moves left - if not we end our game & go to the next
        if len(pos_moves) == 0:
            break

        # Any kings? If so, we end the simulation 
        finish = checkMoveKing(pos_moves,game_board)

        if finish != False:
            
            # store more efficient solutions
            if finish in king_solution.keys():
                if len(sequence) < len(king_solution[finish]):
                    # override value with more efficient route
                    sequence.append(finish)
                    king_solution[finish] = sequence
            # always store a new king solution
            if finish not in king_solution.keys():
                sequence.append(finish)
                king_solution[finish] = sequence
            
            break

        # If not, we need to choose a move from candidate moves: shuffle (Future: Check if we have last n moves in `sequence`)
        random.shuffle(pos_moves)

        pos = pos_moves[0]
        i += 1
        if i > 2000:
            break

In [18]:
for key, val in king_solution.items():
    print(f"Found king at {key}")
    print(f"Optimal Path: {val}")

Found king at (0, 4)
Optimal Path: [(6, 4), (5, 3), (4, 2), (5, 0), (4, 1), (2, 2), (1, 4), (0, 5), (0, 4)]
Found king at (0, 0)
Optimal Path: [(6, 4), (5, 3), (4, 2), (5, 0), (4, 1), (2, 2), (1, 0), (0, 2), (1, 1), (1, 2), (0, 0)]
Found king at (1, 6)
Optimal Path: [(6, 4), (5, 3), (4, 2), (5, 0), (4, 1), (2, 0), (1, 0), (0, 2), (1, 3), (2, 5), (3, 5), (1, 6)]
